# Clase Práctica 1: `pandas`, `geopandas`, y Encuesta Origen-Destino de Santiago

Por **Eduardo Graells-Garrido**. <egraells@dcc.uchile.cl>. 

Actualizado: **23 de Marzo de 2023**.

En esta case práctica exploraremos bibliotecas para trabajar con `DataFrames` y experimentaremos con una medida ficticia de "índice de calidad de vida" que compararemos con un índice real.

In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../..") if not "google.colab" in sys.modules else Path("aves_git")

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 96
# esto depende de las fuentes que tengas instaladas en el sistema.
mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"


In [ ]:
from aves.data import eod

viajes = eod.read_trips(EOD_PATH)
personas = eod.read_people(EOD_PATH)


In [ ]:
viajes.columns

In [ ]:
viajes[['ComunaOrigen', 'ComunaDestino', 'ModoAgregado', 'TiempoViaje', 'Periodo', 'FactorLaboralNormal']].sample(5)

In [ ]:
personas.columns

In [ ]:
personas[['AnoNac', 'Sexo', 'Relacion', 'Ocupacion', 'Factor_LaboralNormal']].sample(5)

In [ ]:
set(viajes.columns) & set(personas.columns)

In [ ]:
viajes_persona = viajes.merge(personas)

In [ ]:
viajes_persona[['AnoNac', 'Sexo', 'Relacion', 'Ocupacion', 'Factor_LaboralNormal'] + ['ComunaOrigen', 'ComunaDestino', 'ModoAgregado', 'TiempoViaje', 'Periodo', 'FactorLaboralNormal']].sample(5)

In [ ]:
viajes_persona["PesoLaboral"] = (
    viajes_persona["FactorLaboralNormal"] * viajes_persona["Factor_LaboralNormal"]
)

viajes_persona = viajes_persona[pd.notnull(viajes_persona["PesoLaboral"])]

print(
    "{} viajes expandidos a {}".format(
        len(viajes_persona), int(viajes_persona["PesoLaboral"].sum())
    )
)


In [ ]:
hogares = eod.read_homes(EOD_PATH)
hogares.columns

In [ ]:
data = viajes_persona.merge(hogares[['Hogar', 'Comuna', 'Sector']])
data.sample(3)

In [ ]:
from aves.features.utils import weighted_mean

tiempos_por_comuna = data.groupby(['Comuna', 'Proposito']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PesoLaboral')).unstack()
tiempos_por_comuna

In [ ]:
tiempos_por_comuna = tiempos_por_comuna.apply(lambda x: x.fillna(x.median()))

In [ ]:
from aves.features.utils import standardize_columns
sns.clustermap(tiempos_por_comuna.pipe(standardize_columns), center=0, metric='cosine', figsize=(12, 12))

In [ ]:
indice_tiempos = tiempos_por_comuna.pipe(standardize_columns).mean(axis=1).rename('indice_tiempos')
indice_tiempos.sort_values().plot(kind='barh')

In [ ]:
zones = (
    gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
    .set_index("ID")
)
zones.head()

In [ ]:
zones.join(indice_tiempos, on='NOM_COMUNA').plot(column='indice_tiempos')

In [ ]:
from gensim.utils import deaccent
zones['NOM_COMUNA'] = zones['NOM_COMUNA'].str.replace('Ñ', 'NN').map(deaccent).str.replace('NN', 'Ñ')

In [ ]:
zones.join(indice_tiempos, on='NOM_COMUNA').plot(column='indice_tiempos')

Ahora leeremos un índice real que mide la calidad de vida.

Fuente del índice: https://estudiosurbanos.uc.cl/wp-content/uploads/2022/06/Informe-Ejecutivo-ICVU-2021-1.pdf
Noticia con tablas: https://www.emol.com/noticias/Nacional/2022/06/15/1064139/ranking-icvu-2021-chile.html

In [ ]:
indice_medido = pd.read_html('https://www.emol.com/noticias/Nacional/2022/06/15/1064139/ranking-icvu-2021-chile.html', decimal=',', thousands='.', encoding='utf-8')

In [ ]:
indice_icvu = indice_medido[-1]
indice_icvu['Comuna'] = indice_icvu['Comuna'].str.upper().str.replace('Ñ', 'NN').map(deaccent).str.replace('NN', 'Ñ')
indice_icvu

In [ ]:
zones.join(indice_icvu.set_index('Comuna'), on='NOM_COMUNA').plot(column='Puntaje ICVU')

In [ ]:
indice_icvu.join(indice_tiempos, on='Comuna', how='inner').corr()